# Classificação dados infomoney2

Nesse notebook vamos pegar os dados do arquivo 'infomoney-data-clean.csv' obtidos por meio de web scraping e classificaremos eles por meio do modelo disponível em https://huggingface.co/lucas-leme/FinBERT-PT-BR.

Para isso, começaremos instalando as dependências que precisaremos:

In [1]:
!pip install transformers 
!pip install sentencepiece 

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


Depois, precisamos importar os pacotes que utilizaremos.

In [2]:
import requests
import pandas as pd
import math as m
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-21 16:26:23.784674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-21 16:26:23.893568: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-21 16:26:23.893594: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-21 

Após isso, precisamos ler os dados do nosso arquivo 'infomoney2.csv'. Para isso executamos o comando:

In [23]:
df = pd.read_csv('/work/DadosWebScrapping/infomoney2.csv')
df

,Unnamed: 0,text,title,subtitle,tags,date
0,0,"Neste setor, a receita média por contrato (RPC...","B3 (B3SA3) registra queda de 20,5% no volume n...","Velocidade de giro caiu de 167,3% em março de ...","['Mercados', 'Ações', 'B3', 'Prévia']",2023-04-12 19:22:00
1,1,"Os distratos encerraram o 1T23 a R$ 30,65 milh...",Mitre (MTRE3) tem alta de 58% nas vendas no 1º...,"Velocidade de vendas (VSO) no 1T23 atingiu 12,...","['Mercados', 'Negócios', 'Ações', 'Construtora...",2023-04-12 19:02:00
2,2,A decisão sobre a não incidência do tributo no...,"STF: placar se acirra em 5 a 5 em ação ADC 49,...",Caso trata de créditos de ICMS após Corte dete...,"['Política', 'ICMS', 'STF', 'Varejo']",2023-04-12 18:54:00
3,3,"Na Alea, as vendas líquidas foram de R$ 10,7 m...",Tenda (TEND3) registra vendas líquidas de R$ 6...,"A Tenda lançou 7 empreendimentos no 1T23, tota...","['Mercados', 'Negócios', 'Ações', 'Construção ...",2023-04-12 18:45:00
4,4,"Já nos últimos 12 meses, predominam BDRs de ET...","BDRs de ETFs de ouro, prata e tecnologia se de...",Número de investidores pessoa física tiveram l...,"['Onde Investir', 'BDR', 'BDRs de ETFs', 'Dive...",2023-04-12 18:36:00
...,...,...,...,...,...,...
2015,2015,A S&P decidiu rebaixar o rating do First Repub...,S&P rebaixa rating do First Republic Bank de A...,Agência diz que o risco de saída de depósitos ...,"['Economia', 'Mercados', 'Bancos', 'Estados Un...",2023-03-15 11:35:00
2016,2016,Este é o terceiro aumento mensal consecutivo n...,Índice de confiança das construtoras dos EUA s...,Esse é o 3° aumento mensal consecutivo nos nív...,"['Economia', 'Mercados', 'EUA', 'NAHB']",2023-03-15 11:30:00
2017,2017,NaN,STF decide que Imposto de Renda não deve incid...,Ministros entendem que taxar a transferência d...,"['Minhas Finanças', 'Doação', 'Herança', 'Impo...",2023-03-15 11:11:00
2018,2018,Após o colapso do sistema financeiro de 15 ano...,Descasamento de liquidez pode ser “3º dominó a...,"Em carta a investidores, Fink destacou que é c...","['Onde Investir', 'BlackRock', 'Fundos de Inve...",2023-03-15 11:06:00


Para a classificação dos dados utilizaremos somente os títulos e os subtítulos. Assim para selecionar somente a coluna dos títulos, executamos o comando:

In [4]:
titles = df.loc[:, df.columns == 'title']
titles

,title
0,"B3 (B3SA3) registra queda de 20,5% no volume n..."
1,Mitre (MTRE3) tem alta de 58% nas vendas no 1º...
2,"STF: placar se acirra em 5 a 5 em ação ADC 49,..."
3,Tenda (TEND3) registra vendas líquidas de R$ 6...
4,"BDRs de ETFs de ouro, prata e tecnologia se de..."
...,...
2015,S&P rebaixa rating do First Republic Bank de A...
2016,Índice de confiança das construtoras dos EUA s...
2017,STF decide que Imposto de Renda não deve incid...
2018,Descasamento de liquidez pode ser “3º dominó a...


E para selecionar somente a coluna dos Subtítulos, executamos o comando:

In [5]:
subtitles = df.loc[:, df.columns == 'subtitle']
subtitles

,subtitle
0,"Velocidade de giro caiu de 167,3% em março de ..."
1,"Velocidade de vendas (VSO) no 1T23 atingiu 12,..."
2,Caso trata de créditos de ICMS após Corte dete...
3,"A Tenda lançou 7 empreendimentos no 1T23, tota..."
4,Número de investidores pessoa física tiveram l...
...,...
2015,Agência diz que o risco de saída de depósitos ...
2016,Esse é o 3° aumento mensal consecutivo nos nív...
2017,Ministros entendem que taxar a transferência d...
2018,"Em carta a investidores, Fink destacou que é c..."


Depois disso, precisamos configurar nosso modelo que classificará os textos. Para isso definimos qual é o caminho até os arquivos de nosso modelo:

In [6]:
path = '/work/ModeloHug'

Após isso, passamos o path para as classes AutoModelForSequenceClassification e AutoTokenizer para carregar o modelo pré-treinado e seu tokenizer associado. O tokenizer é o responsável pelas entradas do modelo. 

Além disso,  criamos um pipeline passando seu tipo ("sentiment-analysis") o modelo e o tokenizer. Um pipeline é um objeto que abstrai boa parte dos códigos completos da biblioteca e facilita a utilização do modelo.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(path)

model = AutoModelForSequenceClassification.from_pretrained(path)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Para utilizar o modelo criado, basta digitar:

In [8]:
classifier("Frase que deseja analisar") 

[{'label': 'NEUTRAL', 'score': 0.6552619934082031}]

Exemplo real de uma notícia do mercado financeiro:

In [9]:
print(classifier("Ibovespa atinge recorde histórico e anima investidores"))

[{'label': 'POSITIVE', 'score': 0.7519307136535645}]


Como podemos ver, o resultado mais provável é de que a notícia é positiva. Agora vamos fazer isso para todos os títulos e subtítulos de notícias do nosso banco de dados. 

Começaremos pelos títulos. Para fazer isso criamos uma lista (results) para guardar os resultados do modelo. Depois criamos um laço que percorrerá todos os títulos, fará a classificação dele e adicionará o resultado dessa classificação na lista results. 

In [10]:
resultsTitles = []
for x in range(2020):
    # print(x)
    if type(titles['title'][x]) is str:
        result = (classifier(titles['title'][x]))[0]['label']
        resultsTitles.append(result)
    else:
        resultsTitles.append('NaN')

Fazemos a mesma coisa com os subtítulos:

In [11]:
resultsSubTitles = []
for x in range(2020):
    # print(x)
    if type(subtitles['subtitle'][x]) is str:
        result = (classifier(subtitles['subtitle'][x]))[0]['label']
        resultsSubTitles.append(result)
    else:
        resultsSubTitles.append('NaN')

Após isso, adicionamos mais duas colunas ao nosso banco de dados inicial com os resultados obtidos.

In [24]:
df['classTitles'] = resultsTitles
df['classSubTitles'] = resultsSubTitles
df

,Unnamed: 0,text,title,subtitle,tags,date,classTitles,classSubTitles
0,0,"Neste setor, a receita média por contrato (RPC...","B3 (B3SA3) registra queda de 20,5% no volume n...","Velocidade de giro caiu de 167,3% em março de ...","['Mercados', 'Ações', 'B3', 'Prévia']",2023-04-12 19:22:00,NEGATIVE,POSITIVE
1,1,"Os distratos encerraram o 1T23 a R$ 30,65 milh...",Mitre (MTRE3) tem alta de 58% nas vendas no 1º...,"Velocidade de vendas (VSO) no 1T23 atingiu 12,...","['Mercados', 'Negócios', 'Ações', 'Construtora...",2023-04-12 19:02:00,POSITIVE,POSITIVE
2,2,A decisão sobre a não incidência do tributo no...,"STF: placar se acirra em 5 a 5 em ação ADC 49,...",Caso trata de créditos de ICMS após Corte dete...,"['Política', 'ICMS', 'STF', 'Varejo']",2023-04-12 18:54:00,NEUTRAL,NEGATIVE
3,3,"Na Alea, as vendas líquidas foram de R$ 10,7 m...",Tenda (TEND3) registra vendas líquidas de R$ 6...,"A Tenda lançou 7 empreendimentos no 1T23, tota...","['Mercados', 'Negócios', 'Ações', 'Construção ...",2023-04-12 18:45:00,POSITIVE,POSITIVE
4,4,"Já nos últimos 12 meses, predominam BDRs de ET...","BDRs de ETFs de ouro, prata e tecnologia se de...",Número de investidores pessoa física tiveram l...,"['Onde Investir', 'BDR', 'BDRs de ETFs', 'Dive...",2023-04-12 18:36:00,POSITIVE,NEGATIVE
...,...,...,...,...,...,...,...,...
2015,2015,A S&P decidiu rebaixar o rating do First Repub...,S&P rebaixa rating do First Republic Bank de A...,Agência diz que o risco de saída de depósitos ...,"['Economia', 'Mercados', 'Bancos', 'Estados Un...",2023-03-15 11:35:00,NEGATIVE,NEGATIVE
2016,2016,Este é o terceiro aumento mensal consecutivo n...,Índice de confiança das construtoras dos EUA s...,Esse é o 3° aumento mensal consecutivo nos nív...,"['Economia', 'Mercados', 'EUA', 'NAHB']",2023-03-15 11:30:00,POSITIVE,POSITIVE
2017,2017,NaN,STF decide que Imposto de Renda não deve incid...,Ministros entendem que taxar a transferência d...,"['Minhas Finanças', 'Doação', 'Herança', 'Impo...",2023-03-15 11:11:00,NEUTRAL,NEGATIVE
2018,2018,Após o colapso do sistema financeiro de 15 ano...,Descasamento de liquidez pode ser “3º dominó a...,"Em carta a investidores, Fink destacou que é c...","['Onde Investir', 'BlackRock', 'Fundos de Inve...",2023-03-15 11:06:00,NEGATIVE,NEGATIVE


Para obter maior confiabilidade nos nossos dados, faremos uma comparação da classificação obtida pelo título e subtítulo de uma mesma notícia.

Para fazer isso, começaremos a selecionar as duas colunas referentes as classificações.

In [25]:
classification = df.loc[:,['classTitles', 'classSubTitles']]
classification

,classTitles,classSubTitles
0,NEGATIVE,POSITIVE
1,POSITIVE,POSITIVE
2,NEUTRAL,NEGATIVE
3,POSITIVE,POSITIVE
4,POSITIVE,NEGATIVE
...,...,...
2015,NEGATIVE,NEGATIVE
2016,POSITIVE,POSITIVE
2017,NEUTRAL,NEGATIVE
2018,NEGATIVE,NEGATIVE


Depois criamos um laço que passa por todas as 'linhas' da nossa tabela anterior, compara as duas classificações e guarda essa informação numa lista (resultsMatch). Além disso ela guarda a quantidade de cada resultado (de Miss, Match, NaN).

In [26]:
qtdNaN = 0
qtdMatch = 0
qtdMiss = 0
resultsMatch = []
for x in range(2020):
    if((type(classification['classTitles'][x]) is str) and (type(classification['classSubTitles'][x]) is str)):
        if(classification['classTitles'][x] == "NaN" or classification['classSubTitles'][x] == "NaN"):
            qtdNaN += 1
            resultsMatch.append("NaN")
        elif(classification['classTitles'][x] == classification['classSubTitles'][x]):
            qtdMatch += 1
            resultsMatch.append("Match")
        else:
            qtdMiss += 1
            resultsMatch.append("Miss")
    else:
        qtdNaN += 1
        resultsMatch.append("NaN")
total = qtdMatch + qtdMiss + qtdNaN
tabela = [["","Match", "Miss", "NaN", "Total"], ["Quantidade", qtdMatch , qtdMiss, qtdNaN, total], ["Proporção", "%.2f%c" % ((qtdMatch/total * 100), "%"), "%.2f%c" % ((qtdMiss/total * 100), "%"), "%.2f%c" % ((qtdNaN/total * 100), "%"), "%.0f%c" % ((total/total * 100), "%")]]
matchBanco = pd.DataFrame(tabela)
matchBanco

,0,1,2,3,4
0,,Match,Miss,NaN,Total
1,Quantidade,1152,792,76,2020
2,Proporção,57.03%,39.21%,3.76%,100%


Depois disso, precisamos guardar as informações de match em nosso banco. Para fazer isso adicionamos mais uma coluna em nossa tabela

In [27]:
df['Match'] = resultsMatch
df

,Unnamed: 0,text,title,subtitle,tags,date,classTitles,classSubTitles,Match
0,0,"Neste setor, a receita média por contrato (RPC...","B3 (B3SA3) registra queda de 20,5% no volume n...","Velocidade de giro caiu de 167,3% em março de ...","['Mercados', 'Ações', 'B3', 'Prévia']",2023-04-12 19:22:00,NEGATIVE,POSITIVE,Miss
1,1,"Os distratos encerraram o 1T23 a R$ 30,65 milh...",Mitre (MTRE3) tem alta de 58% nas vendas no 1º...,"Velocidade de vendas (VSO) no 1T23 atingiu 12,...","['Mercados', 'Negócios', 'Ações', 'Construtora...",2023-04-12 19:02:00,POSITIVE,POSITIVE,Match
2,2,A decisão sobre a não incidência do tributo no...,"STF: placar se acirra em 5 a 5 em ação ADC 49,...",Caso trata de créditos de ICMS após Corte dete...,"['Política', 'ICMS', 'STF', 'Varejo']",2023-04-12 18:54:00,NEUTRAL,NEGATIVE,Miss
3,3,"Na Alea, as vendas líquidas foram de R$ 10,7 m...",Tenda (TEND3) registra vendas líquidas de R$ 6...,"A Tenda lançou 7 empreendimentos no 1T23, tota...","['Mercados', 'Negócios', 'Ações', 'Construção ...",2023-04-12 18:45:00,POSITIVE,POSITIVE,Match
4,4,"Já nos últimos 12 meses, predominam BDRs de ET...","BDRs de ETFs de ouro, prata e tecnologia se de...",Número de investidores pessoa física tiveram l...,"['Onde Investir', 'BDR', 'BDRs de ETFs', 'Dive...",2023-04-12 18:36:00,POSITIVE,NEGATIVE,Miss
...,...,...,...,...,...,...,...,...,...
2015,2015,A S&P decidiu rebaixar o rating do First Repub...,S&P rebaixa rating do First Republic Bank de A...,Agência diz que o risco de saída de depósitos ...,"['Economia', 'Mercados', 'Bancos', 'Estados Un...",2023-03-15 11:35:00,NEGATIVE,NEGATIVE,Match
2016,2016,Este é o terceiro aumento mensal consecutivo n...,Índice de confiança das construtoras dos EUA s...,Esse é o 3° aumento mensal consecutivo nos nív...,"['Economia', 'Mercados', 'EUA', 'NAHB']",2023-03-15 11:30:00,POSITIVE,POSITIVE,Match
2017,2017,NaN,STF decide que Imposto de Renda não deve incid...,Ministros entendem que taxar a transferência d...,"['Minhas Finanças', 'Doação', 'Herança', 'Impo...",2023-03-15 11:11:00,NEUTRAL,NEGATIVE,Miss
2018,2018,Após o colapso do sistema financeiro de 15 ano...,Descasamento de liquidez pode ser “3º dominó a...,"Em carta a investidores, Fink destacou que é c...","['Onde Investir', 'BlackRock', 'Fundos de Inve...",2023-03-15 11:06:00,NEGATIVE,NEGATIVE,Match


Por fim, criamos um novo arquivo para salvar o novo banco.

In [28]:
df.to_csv('/work/DadosClassificados/infomoney2CLASSIFIED.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4246ec96-5bda-435e-947c-91f08e9f7576' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>